### Import Modules

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np
import missingno as msno
from pyspark.sql import SparkSession
from azure.storage.blob import ContainerClient
from blob_credentials import facts_sas_token, facts_container, workspace_sas_token, workspace_container
from functions.import_data import import_datasets
from functions.utils import find_first_value
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
import rpy2.robjects.packages as rpackages
from rpy2.robjects.vectors import StrVector

### Installing R modules

In [ ]:
# import R's utility package
utils = rpackages.importr('utils')

# select a mirror for R packages
utils.chooseCRANmirror(ind=1) # select the first mirror in the list

# R package names
packnames = ('RBtest')

# Selectively install what needs to be install.
# We are fancy, just because we can.
names_to_install = [x for x in packnames if not rpackages.isinstalled(x)]
if len(names_to_install) > 0:
    utils.install_packages(StrVector(names_to_install))
    
RBtest = importr('RBtest')

### Spark Session

In [ ]:
myname = "marc-samvath-philippe.vigneron"

spark = SparkSession \
    .builder \
    .appName(f"Test-{myname}") \
    .config("spark.executor.instance", "1") \
    .config("spark.executor.memory","512m") \
    .config('spark.jars.packages',"org.apache.hadoop:hadoop-azure:3.1.1") \
    .config("fs.azure", "org.apache.hadoop.fs.azure.NativeAzureFileSystem") \
    .config("fs.wasbs.impl","org.apache.hadoop.fs.azure.NativeAzureFileSystem") \
    .config(f"fs.azure.sas.{facts_container}.hecdf.blob.core.windows.net", facts_sas_token) \
    .config(f"fs.azure.sas.{workspace_container}.hecdf.blob.core.windows.net", workspace_sas_token) \
    .getOrCreate()

In [ ]:
dataset_challenge = import_datasets()[0]
final_mapping = import_datasets()[1]

### Histogram of columns

The first thing is to see how many columns exist for each asset type.

In [ ]:
ax = sns.countplot(x="Type", data=final_mapping)
plt.xticks(rotation=45)
plt.title('Number of columns for each Asset type')

### Missing Values Visualisation

For each asset type, we check the missing values as a time series - to see where the time series begins(which year) or for that matter how many missing values occur in a row and to assess the sparsity of each row as well.

#### Bonds

In [ ]:
msno.matrix(dataset_challenge[[str(i) for i in range(59)]])
plt.title('Missing Values Bonds',fontsize=20)
plt.xlabel('Asset Columns',fontsize=18)
plt.ylabel('Time series values',fontsize=18)

Many of the time series actually start very late. This means that there is a possibility we need to use other column values in the same row to impute missing values for these columns, as there might not be enough values in the time series itself, alone, to impute the missing values.

#### CDS

In [ ]:
msno.matrix(dataset_challenge[[str(i) for i in range(59,100)]])
plt.title('Missing Values CDS',fontsize=20)
plt.xlabel('Asset Columns',fontsize=18)
plt.ylabel('Time series values',fontsize=18)

For the CDS asset, we notice that almost all columns are populated from the beginning. In addition there seems to be a balance between number of columns that have many missing values vs those that have a few missing values. However, this is a sample of the total number of columns and not all - so the results are not completely conclusive.

#### Commodities

In [ ]:
msno.matrix(dataset_challenge[[str(i) for i in range(735,815)]])
plt.title('Missing Values COMMO_CURVE_FO',fontsize=20)
plt.xlabel('Asset Columns',fontsize=18)
plt.ylabel('Time series values',fontsize=18)

With some time series starting from the beginning and others starting from a later year, this asset type too could benefit from correlations with other columns within the asset type and outside of it. Definitely, 'removing' the missing values or imputing with a constant, is not a very efficient option.

#### Foreign Exchange Rate

In [ ]:
msno.matrix(dataset_challenge[[str(i) for i in range(815,925)]])
plt.title('Missing Values FX Rate',fontsize=20)
plt.xlabel('Asset Columns',fontsize=18)
plt.ylabel('Time series values',fontsize=18)

For this asset type there seem to be many columns that have a lot of rows with consecutive missing values. This might impact the algorithm chosen for imputation.

#### Stocks

In [ ]:
msno.matrix(dataset_challenge[[str(i) for i in range(926,1000)]])
plt.title('Missing Values Stocks',fontsize=20)
plt.xlabel('Asset Columns',fontsize=18)
plt.ylabel('Time series values',fontsize=18)

Stocks, too, as other asset types shows high sparsity. 

#### Foreign Exchange Rate

In [ ]:
msno.matrix(dataset_challenge[[str(i) for i in range(1253,1300)]])
plt.title('Missing Values FX Rate',fontsize=20)
plt.xlabel('Asset Columns',fontsize=18)
plt.ylabel('Time series values',fontsize=18)

It is easily visible that a few columns have fewer nan values as compared to others. There are also a few columns that have chunks of missing values together - rows with consecutive missing nans.

### Heatmap for correlation of columns

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(dataset_challenge[['0','1','2','3','4','5','6','7','8','9','10']].corr(),annot=True)
plt.title('Heatmap for correlation',fontsize=20)

As is quite evident, there is a lot of correlation (positive or negative) observed between different columns which can be exploited for further use.

### Successive NaN values

In [ ]:
def count_max_successive_nan(values):
    """
    How many successive nans in max ?
    Params:
        - values (np.array) : must be ordered by time
    Output: integer
    """
    max_total = 0
    res = 0
    
    for val in values:
        if np.isnan(val):
            res += 1
        elif res!=0:
            total = res
            res = 0
            if total > max_total:
                max_total = total
    
    return max_total

In [ ]:
#Gather the maximum successive nan values for each column in a dictionary
max_successive_nans = dict()
for col in dataset_challenge.columns[1:]:
    first_val = find_first_value(dataset_challenge.loc[:,col].to_numpy())
    if first_val != 'NaN':
        values = dataset_challenge.loc[first_val:,col].to_numpy()
    else:
        values = dataset_challenge.loc[:,col].to_numpy()
    max_successive_nans[col] = count_max_successive_nan(values)

# Sorted dictionary
{k: v for k, v in sorted(max_successive_nans.items(), key=lambda item: item[1], reverse=True)}

#Keeping only greater than 500 successive Nans 
data = [(k,v) for k,v in max_successive_nans.items() if v]
df_max_nans = pd.DataFrame.from_records(data, columns=['keys', 'values'])

In [ ]:
df_max_nans.hist(bins=10)
plt.title('Successive Nan values',fontsize=10)
plt.xlabel('Columns',fontsize=10)
plt.ylabel('Number of max successive nan values',fontsize=10)

Most of the columns have 2 successive nan values at maximum. 

### Test for MCAR

In [ ]:
#Pandas to r dataset converter
with localconverter(ro.default_converter + pandas2ri.converter):
    r_dataset_challenge = ro.conversion.py2rpy(dataset_challenge)

In [ ]:
## Collating the test results to check which columns are MCAR, MAR or not missing.

first_rbtest = RBtest.RBtest(r_dataset_challenge)
test_results = np.asarray(first_rbtest)[2].astype(int).astype(str)
test_results[test_results == '-1'] = 'Not Missing'
test_results[test_results == '0'] = 'MCAR'
test_results[test_results == '1'] = 'MAR'
test_results = test_results.tolist()

In [ ]:
##Plotting the missing values MCAR, MAR or Not missing
plt.figure(figsize=(10,7))

ax = sns.countplot(test_results)
ax.set_title('Missing Value Types Count', fontsize=20)
ax.tick_params(axis='both', which='major', labelsize=15)
ax.set_ylabel("Count",fontsize=15)

for p in ax.patches:
    x=p.get_bbox().get_points()[:,0]
    y=p.get_bbox().get_points()[1,1]
    ax.annotate('{:.1f}%'.format(100.*y/len(test_results)), (x.mean(), y), 
            ha='center', va='bottom', fontsize=15) # set the alignment of the text

plt.savefig('Missing_Values.png')
plt.show()

As can be seen above, the test was conducted to see whether data was missing completely at random, missing at random or not missing. According to the plot, most of the data is missing completely at random which opens up various possibilities for imputation and does not restrict us to dependencies.

### Percentage of missing values

In [ ]:
## Creating a different dataset for each asset type
bond = dataset_challenge.iloc[:,0:58]
cds = dataset_challenge.iloc[:,59:733]
commodities = dataset_challenge.iloc[:,734:814]
fxrate = dataset_challenge.iloc[:,815:925]
stock = dataset_challenge.iloc[:,926:1252]
yieldc = dataset_challenge.iloc[:,1253:1503]

In [ ]:
##Percentage of missing values by asset type
missing_asset_type = []
missing_asset_type.append(bond.isna().sum().sum()/(58*2851))
missing_asset_type.append(cds.isna().sum().sum()/((733-59)*2851))
missing_asset_type.append(commodities.isna().sum().sum()/((814-734)*2851))
missing_asset_type.append(fxrate.isna().sum().sum()/((925-815)*2851))
missing_asset_type.append(stock.isna().sum().sum()/((1252-926)*2851))
missing_asset_type.append(yieldc.isna().sum().sum()/((1503-1253)*2851))

In [ ]:
##Percentage of missing values by total columns
missing_total = []
missing_total.append(bond.isna().sum().sum()/(1504*2851))
missing_total.append(cds.isna().sum().sum()/(1504*2851))
missing_total.append(commodities.isna().sum().sum()/(1504*2851))
missing_total.append(fxrate.isna().sum().sum()/(1504*2851))
missing_total.append(stock.isna().sum().sum()/(1504*2851))
missing_total.append(yieldc.isna().sum().sum()/(1504*2851))
xlabels = ['Bond','CDS','commodities','fxrate','stock','yieldc']
x = np.arange(6)

In [ ]:
##Plot of the percentage of missing values
ax = plt.subplot(111)
w = 0.3
asset = ax.bar(x-w, missing_asset_type, width=w, color='b', align='center')
total = ax.bar(x, missing_total, width=w, color='r', align='center')
ax.autoscale(tight=True)
plt.xticks(ticks=x,labels=xlabels)
plt.yticks([0.1,0.2,0.3,0.4,0.5])
plt.legend([asset, total],['%age missing by asset', '%age missing total'])
plt.title('Percentage of missing values')
plt.xlabel('Asset type')
plt.ylabel('Percentage')
plt.show()

- Above is a plot of the percentage of missing values by asset to the percentage of missing values by total number of values. Bonds has the maximum number of missing values by asset and that is perhaps due to the fact that as seen earlier, many of the time series start at a much later year as compared to the time series of other assets.

- In addition, CDS has the highest percentage of missing values by all values, and that is very much credited to the fact that it has the most number of columns as compared to the other assets.

### Distribution of percentage of missing data

In [ ]:
def ts_missing_proportions(data: pd.DataFrame) -> list:
    '''
    input: a dataframe
    output: List
    Description: Creates a list with proportions of missing values
    '''
    props=[]
    for col in data.columns:
        series=data[col]
        first_valid_index=series.first_valid_index()
        ts=series[first_valid_index:]
        length=len(ts)
        nulls=ts.isnull().sum()
        props.append(nulls/length)
    return props

In [ ]:
## Return a list with proportion of missing values
props = ts_missing_proportions(dataset_challenge.drop('Date',axis=1))
##Distribution plot
sns.distplot(props,kde=False,norm_hist=False)
sns.despine(left=True, bottom=True, right=True)
plt.title('Distribution of percentage of missing values')
plt.ylabel('Number of columns')
plt.xlabel('Percentage of missing values')

- 140 assets that have nearly 6% of the data missing. Most of the columns have nearly 4-6% of data missing.
- < 100 assets for more than 10% data missing. However, nearly 50 assets have 0-1% missing values